In [1]:
plant_type=['Apple___Apple_scab','Apple___Black_rot','Apple___Cedar_apple_rust','Apple___healthy',
           'Blueberry___healthy','Cherry_(including_sour)___healthy','Cherry_(including_sour)___Powdery_mildew',
           'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot','Corn_(maize)___Common_rust_','Corn_(maize)___healthy','Corn_(maize)___Northern_Leaf_Blight',
           'Grape___Black_rot','Grape___Esca_(Black_Measles)','Grape___healthy','Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
           'Orange___Haunglongbing_(Citrus_greening)','Peach___Bacterial_spot','Peach___healthy',
           'Pepper,_bell___Bacterial_spot','Pepper,_bell___healthy','Potato___Early_blight','Potato___healthy','Potato___Late_blight',
           'Raspberry___healthy','Soybean___healthy','Squash___Powdery_mildew','Strawberry___healthy','Strawberry___Leaf_scorch',
           'Tomato___Bacterial_spot','Tomato___Early_blight','Tomato___healthy','Tomato___Late_blight','Tomato___Leaf_Mold',
           'Tomato___Septoria_leaf_spot','Tomato___Spider_mites Two-spotted_spider_mite','Tomato___Target_Spot','Tomato___Tomato_mosaic_virus','Tomato___Tomato_Yellow_Leaf_Curl_Virus']
print (len(plant_type))

38


In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [3]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

In [11]:
# 影像大小
IMAGE_SIZE = (224, 224)

# 影像類別數
NUM_CLASSES = 38

# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 8

# 凍結網路層數
FREEZE_LAYERS = 2

# Epoch 數
NUM_EPOCHS = 20

# 模型輸出儲存的檔案
WEIGHTS_FINAL = 'plant-segment-model-resnet50-final.h5'

In [12]:
# 透過 data augmentation 產生訓練與驗證用的影像資料
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_batches = train_datagen.flow_from_directory('./PlantVillage-Dataset/raw/segmented-測試',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

valid_datagen = ImageDataGenerator()
valid_batches = valid_datagen.flow_from_directory('./PlantVillage-Dataset/raw/segmented-驗證',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE)

Found 27144 images belonging to 38 classes.
Found 32572 images belonging to 38 classes.


In [13]:
# 輸出各類別的索引值
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

# 以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))

Class #0 = Apple___Apple_scab
Class #1 = Apple___Black_rot
Class #2 = Apple___Cedar_apple_rust
Class #3 = Apple___healthy
Class #4 = Blueberry___healthy
Class #5 = Cherry_(including_sour)___Powdery_mildew
Class #6 = Cherry_(including_sour)___healthy
Class #7 = Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
Class #8 = Corn_(maize)___Common_rust_
Class #9 = Corn_(maize)___Northern_Leaf_Blight
Class #10 = Corn_(maize)___healthy
Class #11 = Grape___Black_rot
Class #12 = Grape___Esca_(Black_Measles)
Class #13 = Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
Class #14 = Grape___healthy
Class #15 = Orange___Haunglongbing_(Citrus_greening)
Class #16 = Peach___Bacterial_spot
Class #17 = Peach___healthy
Class #18 = Pepper,_bell___Bacterial_spot
Class #19 = Pepper,_bell___healthy
Class #20 = Potato___Early_blight
Class #21 = Potato___Late_blight
Class #22 = Potato___healthy
Class #23 = Raspberry___healthy
Class #24 = Soybean___healthy
Class #25 = Squash___Powdery_mildew
Class #26 = Strawberry___L

/home/awinlab/.local/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [14]:
x = net.output
x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)

In [15]:
# 設定凍結與要進行訓練的網路層
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True

In [16]:
# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])

# 輸出整個網路結構
print(net_final.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [17]:
# 訓練模型
net_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS)

# 儲存訓練好的模型
net_final.save(WEIGHTS_FINAL)

Epoch 1/20
3393/3393 [==============================] - 577s 170ms/step - loss: 1.6968 - acc: 0.6075 - val_loss: 0.2466 - val_acc: 0.9238
Epoch 2/20
3393/3393 [==============================] - 564s 166ms/step - loss: 0.5608 - acc: 0.8414 - val_loss: 0.1612 - val_acc: 0.9507
Epoch 3/20
3393/3393 [==============================] - 565s 166ms/step - loss: 0.3738 - acc: 0.8928 - val_loss: 0.1066 - val_acc: 0.9665
Epoch 4/20
3393/3393 [==============================] - 565s 166ms/step - loss: 0.2766 - acc: 0.9176 - val_loss: 0.0908 - val_acc: 0.9726
Epoch 5/20
3393/3393 [==============================] - 565s 166ms/step - loss: 0.2172 - acc: 0.9348 - val_loss: 0.0596 - val_acc: 0.9823
Epoch 6/20
3393/3393 [==============================] - 565s 166ms/step - loss: 0.1749 - acc: 0.9468 - val_loss: 0.0519 - val_acc: 0.9854
Epoch 7/20
3393/3393 [==============================] - 565s 166ms/step - loss: 0.1508 - acc: 0.9538 - val_loss: 0.0423 - val_acc: 0.9868
Epoch 8/20
3393/3393 [============

In [31]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.preprocessing import image
import sys
import numpy as np

In [35]:
plant_type=['Apple___Apple_scab','Apple___Black_rot','Apple___Cedar_apple_rust','Apple___healthy','background',
           'Blueberry___healthy','Cherry_(including_sour)___healthy','Cherry_(including_sour)___Powdery_mildew',
           'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot','Corn_(maize)___Common_rust_','Corn_(maize)___healthy','Corn_(maize)___Northern_Leaf_Blight',
           'Grape___Black_rot','Grape___Esca_(Black_Measles)','Grape___healthy','Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
           'Orange___Haunglongbing_(Citrus_greening)','Peach___Bacterial_spot','Peach___healthy',
           'Pepper,_bell___Bacterial_spot','Pepper,_bell___healthy','Potato___Early_blight','Potato___healthy','Potato___Late_blight',
           'Raspberry___healthy','Soybean___healthy','Squash___Powdery_mildew','Strawberry___healthy','Strawberry___Leaf_scorch',
           'Tomato___Bacterial_spot','Tomato___Early_blight','Tomato___healthy','Tomato___Late_blight','Tomato___Leaf_Mold',
           'Tomato___Septoria_leaf_spot','Tomato___Spider_mites Two-spotted_spider_mite','Tomato___Target_Spot','Tomato___Tomato_mosaic_virus','Tomato___Tomato_Yellow_Leaf_Curl_Virus']
print (len(plant_type))

39


In [36]:
import os

# 載入訓練好的模型
net = load_model('../model-resnet50-final.h5')


In [33]:
#os.chdir('./orange/')
print (os.getcwd())


/home/awinlab/Documents/cc/orange


In [37]:
# 從參數讀取圖檔路徑
fileDir = './'
pathDir = os.listdir(fileDir)
for f in pathDir:
    img = image.load_img(f, target_size=(224, 224))
    if img is None:
        continue
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    pred = net.predict(x)[0]
    top_inds = pred.argsort()[::-1][:5]
    print(f)
    for i in top_inds:
        print('    {:.3f}  {}'.format(pred[i], plant_type[i]))

o1.png
    0.992  Orange___Haunglongbing_(Citrus_greening)
    0.006  Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
    0.002  Tomato___Spider_mites Two-spotted_spider_mite
    0.000  Tomato___Early_blight
    0.000  Apple___Apple_scab
o8.png
    1.000  Apple___healthy
    0.000  Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
    0.000  Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
    0.000  Cherry_(including_sour)___healthy
    0.000  Orange___Haunglongbing_(Citrus_greening)
o3.png
    0.999  Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
    0.001  Tomato___Early_blight
    0.000  Tomato___Spider_mites Two-spotted_spider_mite
    0.000  Apple___Apple_scab
    0.000  Tomato___Tomato_Yellow_Leaf_Curl_Virus
o9.png
    0.999  Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
    0.001  Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
    0.000  Soybean___healthy
    0.000  Apple___Apple_scab
    0.000  Apple___healthy
o2.png
    1.000  Orange___Haunglongbing_(Citrus_greening)
    0.000  Grape___Le